# 午後の紅茶さん_Mistral


RTX-4090 実行: 2024/9/14

In [1]:
!nvidia-smi

Sun Sep 15 08:33:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090 L...    On | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P8                5W /  55W|    134MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%cd ../
%cd SadTalker
# %ls

/home/atsu/llamaindex
/home/atsu/llamaindex/SadTalker


/home/atsu/anaconda3/envs/llamaindex/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# # Whisperのinstal --> 一度だけ実行
# !pip install git+https://github.com/openai/whisper.git

In [4]:
import uuid # 追加
import re
import os
import glob
import random
import time
import gradio as gr
import io
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import openai
from huggingface_hub import login
import whisper

In [5]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext

使用した画像を削除してしまうので以下のコードをコメントアウトした。  
shutil.move(source_image, input_dir) 

In [6]:
# SadTalker用
# from src.gradio_demo2 import SadTalker
from inference6 import make_video # face_enhancer, background_enhancer 対応

In [7]:
# VOICEVOX 用
from pathlib import Path
import voicevox_core
from voicevox_core import AccelerationMode, AudioQuery, VoicevoxCore
from playsound import playsound
from pydub import AudioSegment
# from IPython.display import Audio

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [8]:
# 基本パラメータ
model_id = "tokyotech-llm/Swallow-MS-7b-instruct-v0.1"
peft_name = "../付喪神ジェネレータ/QLoRA_models/tea_mistral"

In [9]:
# OpenAI API ログイン
from openai import OpenAI
with open('../API_key/OpenAI_API_key.txt', 'r') as f:
    key = f.read()
os.environ["OPENAI_API_KEY"] = key
openai.api_key = key
client = OpenAI()

# Gradioを用いたWebアプリ作成

In [10]:
# 音声からテキストへの変換関数（新バージョン）
# https://platform.openai.com/docs/guides/speech-to-text/quickstart
def speech_to_text_API(input_audio):
    audio_file = open(input_audio, "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_file,
    )

In [11]:
# VOICEVOX によるテキストから音声変換
def VOICEVOX(text, out='output.wav', SPEAKER_ID=2):
    open_jtalk_dict_dir = './open_jtalk_dic_utf_8-1.11'
    acceleration_mode = AccelerationMode.AUTO
    core = VoicevoxCore(
        acceleration_mode=acceleration_mode, open_jtalk_dict_dir=open_jtalk_dict_dir
    )
    core.load_model(SPEAKER_ID)
    audio_query = core.audio_query(text, SPEAKER_ID)
    wav = core.synthesis(audio_query, SPEAKER_ID)
    out_byte = Path(out) # 追加
    out_byte.write_bytes(wav)
    return out

In [12]:
# モデルとトークナイザの準備
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# モデルの設定
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config, # 量子化
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
)
# tokenizerの設定
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding_side="right",
    add_eos_token=True
)
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
%%time
# ファインチューニングモデルの作成
model = PeftModel.from_pretrained(base_model, peft_name)

CPU times: user 3.27 s, sys: 349 ms, total: 3.62 s
Wall time: 1.45 s


In [14]:
# 文章生成関数（EZO用）--> Llama3.1でもこのコードは有効だと分かった
def generate(prompt, max_tokens=256, temperature=1, top_p=0.9, top_k=30):

    prompt = f"""system prompt:あなたは午後の紅茶を飲むユーザーと対話するAIです。お嬢さま口調で対話してください。短く返答してください。
    user prompt: {prompt}"""
    
    messages = [
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(model.device),
    )
    response = outputs[0][input_ids.shape[-1]:]
    response_text = tokenizer.decode(response, skip_special_tokens=True)
    return response_text


In [18]:
def overall_response(input_mode, input_text, input_audio, image, voice, size, preprocess, face_enhancer, bkgnd_enhancer, 
                     max_tokens, temperature, top_p, top_k):
    
    # GPUメモリをリセット
    torch.cuda.empty_cache()
    
    if input_mode == "Text":
        input_text = input_text
    else:
        input_text = speech_to_text_API(input_audio)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # 応答
    code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠！｀ 　＋￥％？ξﾟ⊿abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789]')
    response_text = code_regex.sub('', generate(input_text, max_tokens, temperature, top_p, top_k)).replace("\n","")
    print("origilal response:", response_text) 
    
    print("===============================")
    print("processed response text: " + response_text)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # 返答を音声に
    out = "../付喪神ジェネレータ/results/output.wav"
    if voice=="四国めたんノーマル":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=2) #四国めたんノーマル
    elif voice=="四国めたんあまあま":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=0) #四国めたんあまあま
    elif voice=="四国めたんツンツン":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=6) #四国めたんツンツン
    elif voice=="四国めたんセクシー":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=4) #四国めたんセクシー
    elif voice=="波音リツ":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=9) #波音リツノーマル 
    elif voice=="冥鳴ひまり":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=14) #冥鳴ひまりノーマル
    elif voice=="雨晴はう":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=10) #雨晴はうノーマル
    elif voice=="春日部つむぎ":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=8) #春日部つむぎノーマル
    elif voice=="九州そらノーマル":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=16) #九州そらノーマル
    elif voice=="九州そらあまあま":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=15) #九州そらあまあま
    elif voice=="九州そらツンツン":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=18) #九州そらツンツン
    elif voice=="九州そらセクシー":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=17) #九州そらセクシー
    elif voice=="九州そらささやき":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=19) #九州そらささやき
    
    print("Sound generation finished.") # ターミナルに表示
    print(response_audio)

    # 画像パスを引数に変更
    image_path = image
    print(image_path)
    
    # 追加のデバッグメッセージ
    print(f"Selected image path: {image_path}")
    if not os.path.isfile(image_path):
        raise ValueError(f"Image file does not exist: {image_path}")
    else:
        print(f"Image file exists: {image_path}")

    result_dir = '../付喪神ジェネレータ/results'
    
    # ファイルパスのデバッグメッセージを追加
    print(f"Pic path before test call: {os.path.join(result_dir, str(uuid.uuid4()), 'input', os.path.basename(image_path))}")

    responce_video = make_video(image_path=image_path,
                                audio_path=response_audio,
                                size=size,
                                preprocess=preprocess,
                                enhancer=face_enhancer,
                                background_enhancer=bkgnd_enhancer,
                                result_dir=result_dir,
                                )
    
    print("Video generation finished.") # ターミナルに表示
    print(responce_video)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # return rag_result, responce_video, voice, image_path
    return responce_video, voice, image_path

In [19]:
# Blockの定義
# https://note.com/npaka/n/n2a5112208b8d

# CSSを使用してボタンのスタイルをカスタマイズ
css = """
#custom-button {
    background: #ee7800; /* ボタンの背景色 */
    color: white; /* ボタンのテキスト色 */
    border: none;
    text-align: center;
    font-size: 20px;
    cursor: pointer;
}
"""

with gr.Blocks(css=css) as app:

    # Markdow
    gr.Markdown("""# 超未来の付喪神ジェネレータ　午後の紅茶さん""")

    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            with gr.Row():
                with gr.Column():
                    with gr.Row():
                        with gr.Column():
                            input_mode = gr.Radio(["Text","Voice"], label="Input mode", value="Text")
                            input_text = gr.Textbox(placeholder="Please write the input.", label="input text",)
                            record = gr.Audio(sources=["microphone"], label="Input audio", type="filepath")
                            voice = gr.Dropdown(["random", "四国めたんノーマル", "四国めたんあまあま", "四国めたんツンツン", "四国めたんセクシー",
                                                 "波音リツ", "冥鳴ひまり", "雨晴はう", "春日部つむぎ",
                                                 "九州そらノーマル", "九州そらあまあま", "九州そらツンツン",
                                                 "九州そらセクシー", "九州そらささやき"],
                                                 label="voice", value="春日部つむぎ")
                            clear = gr.ClearButton(components=[record])
                        with gr.Column():
                            image = gr.Image(type='filepath') #画像のpathを返す
                            clear = gr.ClearButton(components=[image])
            with gr.Row():
                btn = gr.Button("Response", elem_id="custom-button") #ボタンにIDを指定
            with gr.Row():
                with gr.Column(scale=1):                        
                    with gr.Row():
                        gr.Markdown("LLM congig")
                        max_tokens = gr.Dropdown([32, 64, 128, 256, 512, 1024, 2048, 4096, 8192], label="max tokens", value=64)
                        temperature = gr.Slider(minimum=0.01, maximum=1, value=1, step=0.01, label="temperature", interactive=True)
                        top_p = gr.Slider(minimum=0, maximum=1, value=0.9, step=0.01, label="top_p", interactive=True)
                        top_k = gr.Slider(minimum=1, maximum=100, value=30, step=1, label="top_k", interactive=True)
                with gr.Column(scale=1):
                    gr.Markdown("SadTalker congig")
                    with gr.Row():
                        preprocess = gr.Dropdown(["crop", "extcrop", "resize", "full", "extfull"], label="preprocess", value="extcrop")
                        size = gr.Dropdown([256, 512], label="image size", value=256)
                        face_enhancer = gr.Dropdown([None, "gfpgan", "RestoreFormer"], label="face enhancer", value=None)
                        bkgnd_enhancer = gr.Dropdown([None, "realesrgan"], label="background enhancer", value=None)
            with gr.Row():
                # rag_result = gr.Text(label="RAG result")
                agent_voice = gr.Text(label="Voice of agent")
                agent_image = gr.Text(label="Image of agent")                
        with gr.Column(scale=1, min_width=300):
            response_video = gr.Video(label="Response video")
            # イベントリスナー
            btn.click(fn=overall_response,
                      inputs=[input_mode, input_text, record, image, voice, 
                              size, preprocess, face_enhancer, bkgnd_enhancer, max_tokens, temperature, top_p, top_k],                   
                      outputs=[response_video, agent_voice, agent_image]
                      )

In [ ]:
# Webアプリを起動
app.launch(share=True, debug=True) # share=Trueで一時的に公開される

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://1c210eb8558bc5b4fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


origilal response: ワタクシ思いますの、午後の紅茶、いい響きですわねー。お茶を淹れるためのお道具は、ワタクシのコレクションの中で最も愛着のあるものの一つでごわー。淹れるお茶の種類は、その日の気分
processed response text: ワタクシ思いますの、午後の紅茶、いい響きですわねー。お茶を淹れるためのお道具は、ワタクシのコレクションの中で最も愛着のあるものの一つでごわー。淹れるお茶の種類は、その日の気分
Sound generation finished.
../付喪神ジェネレータ/results/output.wav
/tmp/gradio/a1a1932be42492edbad21dde28837a48d1d1462e/tea_girl.png
Selected image path: /tmp/gradio/a1a1932be42492edbad21dde28837a48d1d1462e/tea_girl.png
Image file exists: /tmp/gradio/a1a1932be42492edbad21dde28837a48d1d1462e/tea_girl.png
Pic path before test call: ../付喪神ジェネレータ/results/c9e55da5-e88c-4769-afc2-6a679a5a0621/input/tea_girl.png
using safetensor as default
3DMM Extraction for source image


Face Renderer:: 100%|█████████████████████████████████| 176/176 [02:15<00:00,  1.30it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The generated video is named ../付喪神ジェネレータ/results/2024_09_15_08.41.35/tea_girl##output.mp4
Video generation finished.
../付喪神ジェネレータ/results/2024_09_15_08.41.35.mp4
